In [20]:
import scipy.misc
import imageio
import numpy as np
import cv2

original = scipy.misc.imread("/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/foliage/001.png", mode = "YCbCr")
LR = scipy.misc.imresize(original, (160, 240), interp = 'bicubic')
bicubic = scipy.misc.imresize(LR, (480, 720), interp = 'bicubic')

Y = np.split(original, 3, axis=2)[0]
Cb = np.split(original, 3, axis=2)[1]
Cr = np.split(original, 3, axis=2)[2]
img_ycbcr = np.dstack((Y, Cr, Cb))
print("shape:",Y.shape,Cb.shape,Cr.shape,img_ycbcr.shape)
img_rgb = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCrCb2RGB)
imageio.imwrite("test_original.png", img_rgb)

Y_LR = np.split(LR, 3, axis=2)[0]
Cb_LR = np.split(LR, 3, axis=2)[1]
Cr_LR = np.split(LR, 3, axis=2)[2]

#bicubic
Y_bicubic = np.split(bicubic, 3, axis=2)[0]
Cb_bicubic = np.split(bicubic, 3, axis=2)[1]
Cr_bicubic = np.split(bicubic, 3, axis=2)[2]
img_ycbcr = np.dstack((Y_bicubic, Cr_bicubic, Cb_bicubic))
#print("shape:",Y.shape,Cb.shape,Cr.shape,img_ycbcr.shape)
img_rgb = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCrCb2RGB)
imageio.imwrite("test_bicubic.png", img_rgb)

scale = 3
# upscale Cb and Cr
Y_repeat = Y_LR.repeat(scale, axis = 0).repeat(scale, axis = 1)
Cb_repeat = Cb_LR.repeat(scale, axis = 0).repeat(scale, axis = 1)
Cr_repeat = Cr_LR.repeat(scale, axis = 0).repeat(scale, axis = 1)
# stack and save
#img_ycbcr = np.dstack((Y_repeat, Cr_repeat, Cb_repeat))
img_ycbcr = np.dstack((Y_repeat, Cr_bicubic, Cb_bicubic))
#print("shape:",Y.shape,Cb.shape,Cr.shape,img_ycbcr.shape)
img_rgb = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCrCb2RGB)
imageio.imwrite("test_repeat.png", img_rgb)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


shape: (480, 720, 1) (480, 720, 1) (480, 720, 1) (480, 720, 3)


In [4]:
import scipy.misc
import imageio
import numpy as np
import cv2

original = scipy.misc.imread("/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/foliage/001.png", mode = "RGB")
flipped_0 = np.flip(original, axis = 0)
flipped_1 = np.flip(original, axis = 1)
rotated = np.rot90(original)
print(flipped.shape)
print(rotated.shape)

imageio.imwrite("test_original.png", original)
imageio.imwrite("test_flip_0.png", flipped_0)
imageio.imwrite("test_flip_1.png", flipped_1)
imageio.imwrite("test_rotated.png", rotated)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


(480, 720, 3)
(720, 480, 3)


In [2]:
from warp import *
%reload_ext autoreload
%autoreload 2
filename = "/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/calendar/001.png"
img = scipy.misc.imread(filename, mode = "RGB")
img2 = np.expand_dims(img, axis = 0)
img3 = np.concatenate([img2, img2], axis = 0)
#print(img3.shape)
#print(img.shape)
num_batch = 1
mappings = np.zeros([num_batch,img.shape[0], img.shape[1], 2])
for i in range(0, num_batch):
    for j in range(0, img.shape[0]):
        for k in range(0, img.shape[1]):
            #mappings[i,j,k] = [2.0*j/(img.shape[0]-1) -1 , 2.0*k/(img.shape[1]-1)-1]
            mappings[i,j,k] = [0,1]
            
mappings_x = np.zeros([num_batch,img.shape[0], img.shape[1], 1])
mappings_y = np.zeros([num_batch,img.shape[0], img.shape[1], 1])
for i in range(0, num_batch):
    for j in range(0, img.shape[0]):
        for k in range(0, img.shape[1]):
            mappings_x[i,j,k] = 2.0*j/(img.shape[0]-1) -1
            mappings_y[i,j,k] = 2.0*k/(img.shape[1]-1) -1
            
#transformed, index, base, x0, y0, coords, x_coords, y_coords = batch_warp2d(img2, mappings)
transformed, index, base, x0, y0, x_coords, y_coords = batch_warp2d_2_test(img2, mappings_x, mappings_y)
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
x = transformed.eval(session=sess)
y = index.eval(session = sess)
z = base.eval(session = sess)
w = x0.eval(session = sess)
ww = y0.eval(session = sess)
#u = coords.eval(session = sess)
v = x_coords.eval(session = sess)
t = y_coords.eval(session = sess)
print('output:',x[0].shape)
#y = np.reshape(y,[576,720])
print('sampled index:',y)
print('base:',z) #base는 결국 1차원 array 안에서 어느 index를 뽑을지임! 즉, batch_index*H*W가 쭉 반복되는 형태.
print('x0:',w, w.shape, 'y0:',ww, ww.shape)
#print('coords shape:',u.shape)
#print(u[0,:,0], u[0,:,1], u[0,:,2])
#print('coords:',u)
print('x_coords:', v, np.sum(np.reshape(mappings_x, [-1])-v))
print('y_coords:', t)
#print(x.shape)
#print(x[0].dtype)
print(np.sum(x[0]-img))
#print(x[0][0,0,:])
#print(img[0,0,:])
#for i in range(0, img.shape[0]):
#    for j in range(0, img.shape[1]):
#        if img[i,j,0] == x[0][0,0,0] and img[i,j,1] == x[0][0,0,1] and img[i,j,2] == x[0][0,0,2]:
#            print("match!",i,j)
imageio.imwrite("original.png", img)
imageio.imwrite("warped1.png", x[0])
#imageio.imwrite("warped2.png", x[1])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """


output: (576, 720, 3)
sampled index: [     0      0      1 ... 414717 414718 414719]
base: [0 0 0 ... 0 0 0]
x0: [  0   0   0 ... 575 575 575] (414720,) y0: [  0   0   1 ... 717 718 719] (414720,)
x_coords: [-1. -1. -1. ...  1.  1.  1.] 0.0
y_coords: [-1.         -0.99721836 -0.99443672 ...  0.99443672  0.99721836
  1.        ]
-7.702376


/home/johnyi/.local/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 255.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


In [75]:
import imageio
#filename = "/home/johnyi/deeplearning/research/SISR_Datasets/train/DIV2K/0001.png"
filename = "/home/johnyi/deeplearning/research/VSR_Datasets/test/vid4/calendar/001.png"
img = scipy.misc.imread(filename, mode = "RGB")
img2 = np.expand_dims(img, axis = 0)
print(img.shape)

indexes_batch = np.zeros([1, img.shape[0], img.shape[1]] ,dtype = 'int32')
indexes_x = np.zeros([1, img.shape[0], img.shape[1]] ,dtype = 'int32')
indexes_y = np.zeros([1, img.shape[0], img.shape[1]] ,dtype = 'int32')
for i in range(0, 1):
    for j in range(0, img.shape[0]):
        for k in range(0, img.shape[1]):
            indexes_batch[i,j,k] = i
            indexes_x[i,j,k] = j
            indexes_y[i,j,k] = k

tf.reset_default_graph()
in_img = tf.placeholder(tf.float32, [1, img.shape[0], img.shape[1], 3], name='input_t0') 
base_batch = tf.Variable(indexes_batch, name = 'flow_batch')
base_x = tf.Variable(indexes_x, name = 'flow_x')
base_y = tf.Variable(indexes_y, name = 'flow_y')

flow_x = tf.multiply(tf.ones_like(base_x, dtype = tf.int32),20)
flow_y = tf.multiply(tf.ones_like(base_y, dtype = tf.int32),20)
base_x = tf.add(base_x, flow_x)
base_x = tf.clip_by_value(base_x, 0, img.shape[0]-1)
base_y = tf.add(base_y, flow_y)
base_y = tf.clip_by_value(base_y, 0, img.shape[1]-1)

base_batch = tf.expand_dims(base_batch, axis =3)
base_x = tf.expand_dims(base_x, axis = 3)
base_y = tf.expand_dims(base_y, axis = 3)
print(base_batch.shape, base_x.shape, base_y.shape)
flow = tf.concat([base_batch, base_x, base_y], axis = 3)

out_img = tf.gather_nd(in_img, flow)
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
start_time = time.time()
out = sess.run([out_img], feed_dict={in_img: img2})
print("runtime:", time.time()-start_time)
print(out[0][0].shape)

imageio.imwrite("original.png", img)
imageio.imwrite("gather_nd.png", out[0][0])
#print(np.sum(img-out[0]))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  after removing the cwd from sys.path.


(576, 720, 3)
(1, 576, 720, 1) (1, 576, 720, 1) (1, 576, 720, 1)
runtime: 0.02254486083984375
(576, 720, 3)


/home/johnyi/.local/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 255.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


In [76]:
img2 = np.expand_dims(img, axis = 0)
start_time = time.time()
out = sess.run([out_img], feed_dict={in_img:img2})
print("runtime:", time.time()-start_time)

runtime: 0.004666805267333984


In [3]:
import imageio
filename = "/home/johnyi/deeplearning/research/SISR_Datasets/train/DIV2K/0001.png"
img = scipy.misc.imread(filename, mode = "RGB")
print(img.shape)
indexes = np.zeros([img.shape[0], img.shape[1], 2] ,dtype = 'int32')
for i in range(0, img.shape[0]):
    for j in range(0, img.shape[1]):
        indexes[i,j,:] = [i,j]
print(indexes.shape)
print(indexes[1,2])
tf.reset_default_graph()
in_img = tf.placeholder(tf.float32, [img.shape[0], img.shape[1], 3], name='input_t0') 
flow = tf.ones_like(indexes, dtype = tf.int32)
flow = tf.add(tf.multiply(flow,100), indexes)
print(flow.shape)
print(flow[:,:,0].shape)
print(flow[:,:,1].shape)
flow_x = tf.expand_dims(tf.clip_by_value(flow[:,:,0], 0, img.shape[0]-1), axis = 2)
flow_y = tf.expand_dims(tf.clip_by_value(flow[:,:,1], 0, img.shape[1]-1), axis = 2)
flow_clipped = tf.concat([flow_x, flow_y], axis = 2)
print(flow.shape)
out_img = tf.gather_nd(in_img, flow_clipped)
out_img2 = tf.gather_nd(in_img, flow)
sess = tf.Session()
#with sess.as_default():
#    xx = out_img.eval(session = sess) 
#    print(xx.shape)
start_time = time.time()
out, out2, flow, flow_clipped = sess.run([out_img, out_img2, flow, flow_clipped], feed_dict={in_img:img})
print(ru)
#print(out[0].shape)
#print(flow)
#print(flow_clipped)
imageio.imwrite("original.png", img)
#imageio.imwrite("gather_nd.png", out[0])
imageio.imwrite("gather_nd.png", out2)
imageio.imwrite("gather_nd_clip.png", out)
#print(np.sum(img-out[0]))
print(np.sum(out-out2))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


(1404, 2040, 3)
(1404, 2040, 2)
[1 2]
(1404, 2040, 2)
(1404, 2040)
(1404, 2040)
(1404, 2040, 2)


/home/johnyi/.local/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float32 to uint8, range [0.0, 254.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


72657704.0


In [2]:
###======================= TEST =============================###
png_rates = []
for i in range(9):
    if i < 9:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/000"+str(i+1)+".png"
    elif i < 99:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/00"+str(i+1)+".png"
    else:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/0"+str(i+1)+".png"
    img = get_imgs_fn(filename)
    raw_size = img.shape[0]*img.shape[1]*img.shape[2]/(1024*1024)
    png_size = os.path.getsize(filename)/(1024*1024)
    #print("img #",str(i+1)," raw:",raw_size,"Mb, png:",png_size,"Mb")
    compression_rate = png_size/raw_size
    #print("img #",str(i+1)," raw:",raw_size,"Mb, png:",png_size,"Mb, compression ratio:",compression_rate)
    png_rates.append(compression_rate)
    imageio.imwrite('/home/johnyi/test/'+str(i+1)+'.jpeg', img)
    filename2 = '/home/johnyi/test/'+str(i+1)+'.jpeg'
    img2 = get_imgs_fn(filename2)
    print("size:",img.shape,", ",img2.shape,"diff:",np.sum(img-img2))
print("total:",len(rates),", avg:",np.mean(png_rates),", std:",np.std(png_rates))

size: (1404, 2040, 3) ,  (1404, 2040, 3) diff: 988722120
size: (1848, 2040, 3) ,  (1848, 2040, 3) diff: 1276398469
size: (1356, 2040, 3) ,  (1356, 2040, 3) diff: 949072637
size: (1344, 2040, 3) ,  (1344, 2040, 3) diff: 767726412
size: (2040, 1608, 3) ,  (2040, 1608, 3) diff: 1133335971
size: (2040, 1356, 3) ,  (2040, 1356, 3) diff: 882044022
size: (1356, 2040, 3) ,  (1356, 2040, 3) diff: 981776399
size: (1356, 2040, 3) ,  (1356, 2040, 3) diff: 951578901
size: (1524, 2040, 3) ,  (1524, 2040, 3) diff: 1058270059


NameError: name 'rates' is not defined

In [18]:
from scipy.misc import imresize
from skimage.transform import resize

#공개된 테스트셋은 scipy.misc.resize의 결과와 같음.
#skimage의 resize는 뭔가 다름...aliasing 옵션인 것 같기도...
#http://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html

for i in range(5):
    if i < 9:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/000"+str(i+1)+".png"
        filename2 = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_LR_bicubic/X2/000"+str(i+1)+"x2.png"
    elif i < 99:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/00"+str(i+1)+".png"
        filename2 = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_LR_bicubic/X2/00"+str(i+1)+"x2.png"
    else:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/0"+str(i+1)+".png"
        filename2 = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_LR_bicubic/X2/0"+str(i+1)+"x2.png"
    img = get_imgs_fn(filename)
    img2 = get_imgs_fn(filename2)
    #img_LR = resize(img, (img2.shape[0], img2.shape[1]))
    img_LR2 = imresize(img, (img2.shape[0], img2.shape[1]), interp='bicubic')
    imageio.imwrite('./testset.png', img2)
    #imageio.imwrite('./2.png', img_LR)
    imageio.imwrite('./mine.png', img_LR2)
    print("Image ",str(i+1),", HR: ",img.shape,", LR: ",img2.shape)
    #print("diff:",np.sum(img_LR-img_LR2))
    #print("diff:",np.sum(img2-img_LR))
    print("diff:",np.sum(img2-img_LR2))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()


Image  1 , HR:  (1404, 2040, 3) , LR:  (702, 1020, 3)
diff: 56403253
Image  2 , HR:  (1848, 2040, 3) , LR:  (924, 1020, 3)
diff: 74082850
Image  3 , HR:  (1356, 2040, 3) , LR:  (678, 1020, 3)
diff: 54710959
Image  4 , HR:  (1344, 2040, 3) , LR:  (672, 1020, 3)
diff: 48104983
Image  5 , HR:  (2040, 1608, 3) , LR:  (1020, 804, 3)
diff: 66534107


In [2]:
from scipy.misc import imresize
#from skimage.transform import resize
import numpy as np
from utils import *

#공개된 테스트셋은 scipy.misc.resize의 결과와 같음.
#skimage의 resize는 뭔가 다름...aliasing 옵션인 것 같기도...
#http://scikit-image.org/docs/dev/auto_examples/transform/plot_rescale.html
imdb = []
for i in range(5):
    if i < 9:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/000"+str(i+1)+".png"
        filename2 = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_LR_bicubic/X2/000"+str(i+1)+"x2.png"
    elif i < 99:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/00"+str(i+1)+".png"
        filename2 = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_LR_bicubic/X2/00"+str(i+1)+"x2.png"
    else:
        filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/0"+str(i+1)+".png"
        filename2 = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_LR_bicubic/X2/0"+str(i+1)+"x2.png"
    img = get_imgs_fn(filename)
    img_LR1 = imresize(img, (int(img.shape[0]/2), int(img.shape[1]/2)), interp='bicubic')
    img_HR1 = imresize(img_LR1, (img.shape[0], img.shape[1]), interp='bicubic')
    img_LR2 = imresize(img, (int(img.shape[0]/4), int(img.shape[1]/4)), interp='bicubic')
    img_HR2 = imresize(img_LR2, (img.shape[0], img.shape[1]), interp='bicubic')
    PSNR = calc_PSNR(img,img)
    PSNR1 = calc_PSNR(img,img_HR1)
    PSNR2 = calc_PSNR(img,img_HR2)
    print("PSNR:",PSNR,",",PSNR1,",",PSNR2 )
    imdb.append(img)
    print(len(imdb))
print(len(imdb))
print(imdb[1].shape)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


PSNR: 100 , 36.217353355738375 , 31.928384330834323
1
PSNR: 100 , 32.36800714737163 , 31.517645112256197
2
PSNR: 100 , 32.21646970246458 , 30.772894358598954
3
PSNR: 100 , 37.37684917428198 , 35.83713181529359
4
PSNR: 100 , 33.11120441626228 , 30.743111864786336
5
5
(1848, 2040, 3)


In [39]:
def get_batch(imdb, start, batch_size, patch_size):
    img_batch = np.zeros([batch_size, patch_size, patch_size, 3])
    for i in range(batch_size):
        img_index = 0
        H = 0
        W = 0
        img_index = (start+i)%len(imdb)
        H = np.random.randint(imdb[img_index].shape[0]-patch_size)
        W = np.random.randint(imdb[img_index].shape[1]-patch_size)
        print("img index:",img_index,", img shape:",imdb[img_index].shape[0],'',imdb[img_index].shape[1],"H,W:",[H,W])
        img_batch[i,:,:,:] = imdb[img_index][H:H+patch_size, W:W+patch_size,:]
    return img_batch

In [45]:
x = get_batch(imdb, 4, 2, 128)
print(x.shape)
imageio.imwrite('./testset1.png', x[0])
imageio.imwrite('./testset2.png', x[1])

img index: 4 , img shape: 2040  1608 H,W: [1021, 1273]
img index: 0 , img shape: 1404  2040 H,W: [640, 1417]
(2, 128, 128, 3)


/home/johnyi/.local/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [0.0, 240.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))
/home/johnyi/.local/lib/python3.5/site-packages/imageio/core/util.py:104: UserWarning: Conversion from float64 to uint8, range [0.0, 198.0]
  'range [{2}, {3}]'.format(dtype_str, out_type.__name__, mi, ma))


In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imageio
from scipy.misc import imresize, imread
import scipy
#im = cv2.cvtColor(bgr, cv2.COLOR_BGR2YCR_CB)
def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    return np.uint8(rgb.dot(xform.T))

def ycbcr2rgb2(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    rgb = rgb.dot(xform.T)
    np.putmask(rgb, rgb > 255, 255)
    np.putmask(rgb, rgb < 0, 0)
    return np.uint8(rgb)

#http://www.pythonexample.com/code/rgb-to-yuv-conversion-formula/
def YUV2RGB( yuv ):
    m = np.array([[ 1.0, 1.0, 1.0],
                 [-0.000007154783816076815, -0.3441331386566162, 1.7720025777816772],
                 [ 1.4019975662231445, -0.7141380310058594 , 0.00001542569043522235] ])
     
    rgb = np.dot(yuv,m)
    rgb[:,:,0]-=179.45477266423404
    rgb[:,:,1]+=135.45870971679688
    rgb[:,:,2]-=226.8183044444304
    return rgb

filename = "/home/johnyi/deeplearning/research/DIV2K/DIV2K_train_HR/0032.png" #12
img_rgb = scipy.misc.imread(filename, mode='RGB')
img_ycbcr = scipy.misc.imread(filename, mode='YCbCr')
'''
#imageio.imwrite('./original.png', img_rgb)
#imageio.imwrite('./converted.png', img_ycbcr)
#img_ycbcr2 = rgb2ycbcr(img_rgb)
img_rgb2 = ycbcr2rgb(img_ycbcr)
img_rgb3 = ycbcr2rgb2(img_ycbcr)
img_rgb4 = YUV2RGB(img_ycbcr)
#img_ycbcr2 = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2YCR_CB)
#img_rgb3 = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCR2BGR_CB)
#print(np.sum(img_ycbcr-img_ycbcr2))
#print(np.sum(img_rgb-img_rgb2))
imageio.imwrite('./original.png', img_rgb)
imageio.imwrite('./converted1.png', img_rgb2)
imageio.imwrite('./converted2.png', img_rgb3)
imageio.imwrite('./converted3.png', img_rgb4)
print(np.sum(img_rgb-img_rgb2))
print(np.sum(img_rgb-img_rgb3))
print(np.sum(img_rgb-img_rgb4))
'''

img_rgb = scipy.misc.imread(filename, mode='RGB')
img_ycbcr = scipy.misc.imread(filename, mode='YCbCr')
img_ycbcr2 = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
img_rgb2 = cv2.cvtColor(img_ycbcr2, cv2.COLOR_YCrCb2RGB)
imageio.imwrite('./original_2.png', img_rgb)
imageio.imwrite('./converted1_2.png', img_rgb2)
print(np.sum(img_rgb-img_rgb2))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:65: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


199674735


In [15]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imageio
from scipy.misc import imresize, imread
import scipy

def calc_PSNR(img1, img2):
    mse = np.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    return 20 * np.log10(255.0 / np.sqrt(mse))

for i in range(0,5):
    print(i)
    filename1 = "/home/johnyi/deeplearning/research/CVPR16-ESPCN-TF/samples/Set5/original_"+str(i)+".png"
    filename2 = "/home/johnyi/deeplearning/research/CVPR16-ESPCN-TF/samples/Set5/bicubic_"+str(i)+".png"
    filename3 = "/home/johnyi/deeplearning/research/CVPR16-ESPCN-TF/samples/Set5/HR_"+str(i)+".png"
    original = scipy.misc.imread(filename1, mode='RGB')
    original_ycbcr = cv2.cvtColor(original, cv2.COLOR_RGB2YCrCb)
    bicubic = scipy.misc.imread(filename2, mode='RGB')
    bicubic_ycbcr = cv2.cvtColor(bicubic, cv2.COLOR_RGB2YCrCb)
    HR = scipy.misc.imread(filename3, mode='RGB')
    HR_ycbcr = cv2.cvtColor(HR, cv2.COLOR_RGB2YCrCb)

    PSNR1 = calc_PSNR(original, bicubic)
    PSNR2 = calc_PSNR(original, HR)
    print("PSNR (RGB):",PSNR1,PSNR2)
    x = 1
    PSNR1 = calc_PSNR(original_ycbcr[:,:,x], bicubic_ycbcr[:,:,x])
    PSNR2 = calc_PSNR(original_ycbcr[:,:,x], HR_ycbcr[:,:,x])
    print("PSNR (Y):",PSNR1,PSNR2)

0
PSNR (RGB): 34.27800618726744 31.494733025452632
PSNR (Y): 46.96568112963063 34.57830181980665
1
PSNR (RGB): 32.83822607929322 33.107700291831506
PSNR (Y): 39.55318870128834 39.42467263709421
2
PSNR (RGB): 30.674152817260687 30.98510846355081
PSNR (Y): 41.489960397392956 39.65093315301964
3
PSNR (RGB): 33.60767265158971 32.893643081645
PSNR (Y): 40.41646133459976 37.980275511698345
4
PSNR (RGB): 32.54961541592668 31.833087933135403
PSNR (Y): 47.77984207609764 36.108165766512464


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [1]:
import scipy.misc
import numpy as np
import cv2
import imageio
from VESPCN_utils import *
from utils import *

%reload_ext autoreload
%autoreload 2
'''
filename1 = "/home/johnyi/Downloads/001.png"
filename2 = "/home/johnyi/Downloads/original_0_x3.png"
    
img1 = scipy.misc.imread(filename1, mode='RGB')
img2 = scipy.misc.imread(filename2, mode='RGB')
print(np.sum(img1-img2))

img3 = scipy.misc.imread(filename2, mode='YCbCr')
Y = np.split(img3, 3, axis=2)[0]
Cb = np.split(img3, 3, axis=2)[1]
Cr = np.split(img3, 3, axis=2)[2]
img_ycbcr = np.dstack((Y, Cr, Cb))
img_rgb = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCrCb2RGB)
imageio.imwrite("/home/johnyi/Downloads/out.png", img_rgb)

filename3 = "/home/johnyi/Downloads/out.png"
img4 = scipy.misc.imread(filename3, mode='RGB')
print(np.sum(img1-img4))
'''
_, batch_LR, batch_HR = get_testbatch_VSR("calendar", 3, 0, 1, 3, "YCbCr")
print(batch_LR.shape, batch_HR.shape)

filename1 = "/home/johnyi/Downloads/001.png"
img1 = scipy.misc.imread(filename1, mode='YCbCr')
print(img1.dtype)

print(np.sum(img1-np.squeeze(batch_HR)))

HR_temp =  np.copy(batch_HR)
batch_HR_Y = np.split(HR_temp, 3, axis=3)[0]
batch_HR_Cb = np.split(HR_temp, 3, axis=3)[1]
batch_HR_Cr = np.split(HR_temp, 3, axis=3)[2]
img_ycbcr = np.dstack((batch_HR_Y[0,:,:,:] , batch_HR_Cr[0,:,:,:], batch_HR_Cb[0,:,:,:]))
img_ycbcr = img_ycbcr#.astype('uint8')
print(img_ycbcr.shape)
print(np.sum(img_ycbcr-np.squeeze(batch_HR)))
print(np.sum(img_ycbcr-img1))
img_rgb = cv2.cvtColor(img_ycbcr, cv2.COLOR_YCrCb2RGB)
path = "./xx_"+str(0)+"_x"+str(3)+".png"
imageio.imwrite(path, img_rgb)
#print(batch_HR_Y[0,:,:,:].shape, batch_HR_Cb[0,:,:,:].shape)
#save_ycbcr_img(batch_HR_Y[0,:,:,:], batch_HR_Cb[0,:,:,:], batch_HR_Cr[0,:,:,:], 1, path)

(1, 192, 240, 3, 3) (1, 576, 720, 3)
uint8
0
(576, 720, 3)
100264448
100264448


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
